In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('/home/users/datasets/criteo_train1', sep='\t')

In [5]:
data.shape

(10029179, 42)

In [6]:
!ls

README.md  test.ipynb


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV


In [58]:
numeric_features = ["if"+str(i) for i in range(1,14)]
categorical_features = ["cf"+str(i) for i in range(1,27)] + ["day_number"]

fields = ["id", "label"] + numeric_features + categorical_features

In [18]:
fields

['id',
 'label',
 'if1',
 'if2',
 'if3',
 'if4',
 'if5',
 'if6',
 'if7',
 'if8',
 'if9',
 'if10',
 'if11',
 'if12',
 'if13',
 'cf1',
 'cf2',
 'cf3',
 'cf4',
 'cf5',
 'cf6',
 'cf7',
 'cf8',
 'cf9',
 'cf10',
 'cf11',
 'cf12',
 'cf13',
 'cf14',
 'cf15',
 'cf16',
 'cf17',
 'cf18',
 'cf19',
 'cf20',
 'cf21',
 'cf22',
 'cf23',
 'cf24',
 'cf25',
 'cf26',
 'day_number']

In [54]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV

#
# Model pipeline
#

numeric_features = ["if"+str(i) for i in range(1,14)]
categorical_features = ["cf"+str(i) for i in range(1,27)]

fields = ["id", "label"] + numeric_features + categorical_features


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
#    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Now we have a full prediction pipeline.
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('linearregression', LinearRegression())
])

In [64]:
import os, sys
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump
from model import model, fields

In [ ]:
import os, sys
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump

#
# Import model definition
#
from model import model, fields


#
# Logging initialization
#
logging.basicConfig(level=logging.DEBUG)
logging.info("CURRENT_DIR {}".format(os.getcwd()))
logging.info("SCRIPT CALLED AS {}".format(sys.argv[0]))
logging.info("ARGS {}".format(sys.argv[1:]))

#
# Read script arguments
#
try:
  proj_id = sys.argv[1] 
  train_path = sys.argv[2]
except:
  logging.critical("Need to pass both project_id and train dataset path")
  sys.exit(1)


logging.info(f"TRAIN_ID {proj_id}")
logging.info(f"TRAIN_PATH {train_path}")

#
# Read dataset
#
#fields = """doc_id,hotel_name,hotel_url,street,city,state,country,zip,class,price,
#num_reviews,CLEANLINESS,ROOM,SERVICE,LOCATION,VALUE,COMFORT,overall_ratingsource""".replace("\n",'').split(",")

read_table_opts = dict(sep="\t", names=fields, index_col=False)
df = pd.read_table(train_path, **read_table_opts)

#split train/test
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df.iloc[:,-1], test_size=0.33, random_state=42
)

#
# Train the model
#
model.fit(X_train, y_train)

model_score = model.score(X_test, y_test)

logging.info(f"model score: {model_score:.3f}")

# save the model
dump(model, "{}.joblib".format(proj_id))

In [59]:
read_table_opts = dict(sep="\t", names=fields, index_col=False)
df = pd.read_table('/home/users/datasets/criteo_train1', **read_table_opts)

In [ ]:
D = 2 ** 20   # number of weights use for learning
def get_x(csv_row, D):
    x = [0]  # 0 is the index of the bias term
    for key, value in csv_row.items():
        index = int(value + key[1:], 16) % D  # weakest hash ever ;)
        x.append(index)
    return x  # x contains indices of features that have a value of 1

In [1]:
2**20

1048576

In [4]:
import numpy as np
data=np.zeros((2**20, 10000000))

MemoryError: Unable to allocate array with shape (1048576, 10000000) and data type float64

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df.iloc[:,-1], test_size=0.33, random_state=42
)

In [65]:
model.fit(X_train, y_train)

model_score = model.score(X_test, y_test)

MemoryError: Unable to allocate array with shape (6719550, 13) and data type float64

In [57]:
fields

['id',
 'label',
 'if1',
 'if2',
 'if3',
 'if4',
 'if5',
 'if6',
 'if7',
 'if8',
 'if9',
 'if10',
 'if11',
 'if12',
 'if13',
 'cf1',
 'cf2',
 'cf3',
 'cf4',
 'cf5',
 'cf6',
 'cf7',
 'cf8',
 'cf9',
 'cf10',
 'cf11',
 'cf12',
 'cf13',
 'cf14',
 'cf15',
 'cf16',
 'cf17',
 'cf18',
 'cf19',
 'cf20',
 'cf21',
 'cf22',
 'cf23',
 'cf24',
 'cf25',
 'cf26']

In [12]:
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt

In [14]:
train = '/home/users/datasets/criteo_train1'
D = 2 ** 20   # number of weights use for learning
alpha = .1    # learning rate for sgd optimization

In [15]:
def logloss(p, y):
    p = max(min(p, 1. - 10e-12), 10e-12)
    return -log(p) if y == 1. else -log(1. - p)


# B. Apply hash trick of the original csv row
# for simplicity, we treat both integer and categorical features as categorical
# INPUT:
#     csv_row: a csv dictionary, ex: {'Lable': '1', 'I1': '357', 'I2': '', ...}
#     D: the max index that we can hash to
# OUTPUT:
#     x: a list of indices that its value is 1
def get_x(csv_row, D):
    x = [0]  # 0 is the index of the bias term
    for key, value in csv_row.items():
        index = int(value + key[1:], 16) % D  # weakest hash ever ;)
        x.append(index)
    return x  # x contains indices of features that have a value of 1


# C. Get probability estimation on x
# INPUT:
#     x: features
#     w: weights
# OUTPUT:
#     probability of p(y = 1 | x; w)
def get_p(x, w):
    wTx = 0.
    for i in x:  # do wTx
        wTx += w[i] * 1.  # w[i] * x[i], but if i in x we got x[i] = 1.
    return 1. / (1. + exp(-max(min(wTx, 20.), -20.)))  # bounded sigmoid


# D. Update given model
# INPUT:
#     w: weights
#     n: a counter that counts the number of times we encounter a feature
#        this is used for adaptive learning rate
#     x: feature
#     p: prediction of our model
#     y: answer
# OUTPUT:
#     w: updated model
#     n: updated count
def update_w(w, n, x, p, y):
    for i in x:
        # alpha / (sqrt(n) + 1) is the adaptive learning rate heuristic
        # (p - y) * x[i] is the current gradient
        # note that in our case, if i in x then x[i] = 1
        w[i] -= (p - y) * alpha / (sqrt(n[i]) + 1.)
        n[i] += 1.

    return w, n


In [66]:
data.head()

,0,0.1,Unnamed: 2,600,1,342,9,0.2,0.3,18,...,b8170bba,9512c20b,9b6712fa,e611fb63,27fc20dc,Unnamed: 37,201d7147,57e36578,b757e957,day_0
0,1,0,NaN,1424.0,1.0,1291.0,7.0,0.0,0.0,0,...,a1eb1511,9512c20b,fc33311c,f766202e,e1c2c118,NaN,4ffc2977,ff654802,b757e957,day_0
1,2,0,3.0,239.0,3.0,NaN,8.0,0.0,0.0,139,...,b8170bba,cc7a7d35,156cbe87,96fbe197,15562d5d,d3df7183,10c76e2c,30436bfc,e1be5ef2,day_0
2,3,0,11.0,537.0,3.0,NaN,6.0,0.0,0.0,366,...,a1eb1511,cc7a7d35,156cbe87,96fbe197,15562d5d,d3df7183,86c2a370,30436bfc,e1be5ef2,day_0
3,4,0,3.0,124.0,39.0,398.0,3.0,0.0,0.0,1,...,a1eb1511,9512c20b,47849e55,73b3f46d,d994ba60,NaN,d3d5ac76,ff654802,b757e957,day_0
4,5,0,NaN,86.0,0.0,145.0,4.0,0.0,0.0,56,...,b8170bba,0a1d513f,dee25a1b,0683bc6f,20d83ede,NaN,cb957e95,30436bfc,2ccea557,day_0


In [49]:
numeric_features = ["if"+str(i) for i in range(1,14)]
categorical_features = ["cf"+str(i) for i in range(1,27)]

fields = ["id", "label"] + numeric_features + categorical_features

In [50]:
w = [0.] * D  # weights
n = [0.] * D  # number of times we've encountered a feature

# start training a logistic regression model using on pass sgd   , fieldnames=fields
loss = 0.
for t, row in enumerate(open(train)):
    row = dict(zip(fields, row[:-1].split('\t')[:-1]))
    y = 1. if row['label'] == '1' else 0.

    del row['label']  # can't let the model peek the answer
    del row['id']  # we don't need the Id

    # main training procedure
    # step 1, get the hashed features
    x = get_x(row, D)

    # step 2, get prediction
    p = get_p(x, w)

    # for progress validation, useless for learning our model
    loss += logloss(p, y)
    if t % 1000000 == 0 and t > 1:
        print('%s\tencountered: %d\tcurrent logloss: %f' % (
            datetime.now(), t, loss/t))

    # step 3, update model with answer
    w, n = update_w(w, n, x, p, y)

2019-10-27 15:29:53.385781	encountered: 1000000	current logloss: 0.133210
2019-10-27 15:30:50.251588	encountered: 2000000	current logloss: 0.131785
2019-10-27 15:31:45.302644	encountered: 3000000	current logloss: 0.129952
2019-10-27 15:32:39.936304	encountered: 4000000	current logloss: 0.130355
2019-10-27 15:33:33.712891	encountered: 5000000	current logloss: 0.129626
2019-10-27 15:34:27.727154	encountered: 6000000	current logloss: 0.129290
2019-10-27 15:35:25.795361	encountered: 7000000	current logloss: 0.130388
2019-10-27 15:36:26.457302	encountered: 8000000	current logloss: 0.131175
2019-10-27 15:37:33.227829	encountered: 9000000	current logloss: 0.131017
2019-10-27 15:38:37.704944	encountered: 10000000	current logloss: 0.130529


In [53]:
get_x(row, D)

[0,
 2289,
 21234,
 1523,
 244,
 757,
 246,
 247,
 399096,
 6137,
 3856,
 12049,
 69394,
 36627,
 883185,
 950770,
 980979,
 401652,
 723957,
 904182,
 110327,
 705784,
 139257,
 794384,
 204561,
 921362,
 872211,
 229140,
 585493,
 1027862,
 700183,
 765720,
 48921,
 167712,
 384801,
 565026,
 48931,
 884516,
 962341,
 122662]

In [48]:
int('day_0ay_number', 16)

ValueError: invalid literal for int() with base 16: 'day_0ay_number'